In [3]:
# MatchJointerRaw test
import torch

from starry.utils.config import Configuration
from starry.utils.model_factory import loadModel


config = Configuration.createOrLoad('configs/matchjointer1-test.yaml')
model = loadModel(config['model'])


c_time, c_pitch, c_velocity = torch.rand(1, 10) * 10000, torch.randint(128, (1, 10)), torch.rand(1, 10) * 127
s_time, s_pitch, s_velocity = torch.rand(1, 6) * 10000, torch.randint(128, (1, 6)), torch.rand(1, 6) * 127

matching, c_vec, s_vec = model(c_time, c_pitch, c_velocity, s_time, s_pitch, s_velocity)

print('matching:', matching.shape)
print('c_vec:', c_vec.shape)
print('s_vec:', s_vec.shape)


matching: torch.Size([1, 6, 10])
c_vec: torch.Size([1, 6, 128])
s_vec: torch.Size([1, 128, 10])


In [1]:
# dataset
import os
from starry.utils.config import Configuration
from starry.utils.dataset_factory import loadDataset


DATA_DIR = os.environ.get('DATA_DIR')

config = Configuration.create('configs/matchjointer1-test.yaml')
train, val = loadDataset(config, data_dir=DATA_DIR, device='cpu')

it = iter(val)
tensors = next(it)
print('tensors:', tensors)


/home/xunan/work/deep-starry-melody/env/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensors: [{'criterion': {'time': tensor([ 4012.5000,  4307.5000,  4312.5000,  4602.5000,  4637.5000,  4907.5000,
         4912.5000,  5212.5000,  5217.5000,  5537.5000,  5537.5000,  5837.5000,
         5847.5000,  6142.5000,  6157.5000,  6457.5000,  6462.5000,  6757.5000,
         6767.5000,  7062.5000,  7072.5000,  7357.5000,  7377.5000,  7667.5000,
         7687.5000,  7972.5000,  7977.5000,  8267.5000,  8277.5000,  8572.5000,
         8577.5000,  8887.5000,  8887.5000,  9182.5000,  9482.5000,  9487.5000,
         9767.5000, 10072.5000, 10102.5000, 10382.5000, 10677.5000, 10682.5000,
        10982.5000, 11132.5000, 11282.5000, 11302.5000, 11572.5000, 11887.5000,
        11897.5000, 12177.5000, 12507.5000, 12507.5000, 13107.5000, 13107.5000,
        13727.5000, 13737.5000, 14027.5000, 14317.5000, 14332.5000, 14612.5000,
        14917.5000, 15222.5000, 15522.5000, 15527.5000, 15812.5000, 16112.5000,
        16117.5000, 16422.5000, 16712.5000, 16732.5000, 17017.5000, 17307.5000,
       